# Computational Analysis of Sound and Music

# D3 - Deep Learning - Network Architectures

Dr.-Ing. Jakob Abeßer, jakob.abesser@idmt.fraunhofer.de

**Last update:** 28.03.2024

**Outline**

In this notebook, you will learn how to use a convolutional neural network to solve the animal classification task.

In [ ]:
!pip install wget

In [ ]:
import glob
import os
import librosa
import numpy as np
import tensorflow
import matplotlib.pyplot as pl
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
import IPython.display as ipd
import wget
import zipfile

Let's make sure we have the dataset we need

In [ ]:
if not os.path.isfile('animal_sounds.zip'):
    print('Please wait a couple of seconds ...')
    wget.download('https://github.com/machinelistening/machinelistening.github.io/blob/master/animal_sounds.zip?raw=true', 
                      out='animal_sounds.zip', bar=None)
    print('animal_sounds.zip downloaded successfully ...')
else:
    print('Files already exist!')
    
if not os.path.isdir('animal_sounds'):
    print("Let's unzip the file ... ")
    assert os.path.isfile('animal_sounds.zip')
    with zipfile.ZipFile('animal_sounds.zip', 'r') as f:
        # Entpacke alle Inhalte in das angegebene Verzeichnis
        f.extractall('.')
    assert os.path.isdir('animal_sounds')
    print("All done :)")

## Classifying animal sounds with a fully-connected deep neural network

We'll use the animal classification dataset from the lecture on **Machine Learning**

**Reminder**: The dataset used here is a manual selection of 5 examples for 5 animal classes from the https://github.com/karolpiczak/ESC-50 dataset.

Let's use the code from before to load the dataset...

In [ ]:
import glob
dir_dataset = 'animal_sounds'
sub_directories = glob.glob(os.path.join(dir_dataset, '*'))

n_sub = len(sub_directories)
# let's collect the files in each subdirectory
# the folder name is the class name
fn_wav_list = []
class_label = []
file_num_in_class = []

for i in range(n_sub):
    current_class_label = os.path.basename(sub_directories[i])
    current_fn_wav_list = sorted(glob.glob(os.path.join(sub_directories[i], '*.wav')))
    for k, fn_wav in enumerate(current_fn_wav_list):
        fn_wav_list.append(fn_wav)
        class_label.append(current_class_label)
        file_num_in_class.append(k)

n_files = len(class_label)
   
# this vector includes a "counter" for each file within its class, we use it later ...
file_num_in_class = np.array(file_num_in_class)

In [ ]:
unique_classes = sorted(list(set(class_label)))
print("All unique class labels (sorted alphabetically)", unique_classes)
class_id = np.array([unique_classes.index(_) for _ in class_label])


### Feature Extraction

Again, we'll use Mel spectrograms as feature representations.

In [ ]:
def compute_mel_spec_for_audio_file(fn_wav,
                                    n_fft=1024,  
                                    hop_length=441,
                                    fs=22050.,
                                    n_mels=64):
    """ Compute mel spectrogram
    Args:
        fn_wav (str): Audio file name
        n_fft (int): FFT size
        hop_length (int): Hop size in samples
        fs (float): Sample rate in Hz
        n_mels (int): Number of mel bands
    """
    # load audio samples
    x, fs = librosa.load(fn_wav, sr=fs, mono=True)

    # normalize to the audio file to an maximum absolute value of 1
    if np.max(np.abs(x)) > 0:
        x = x / np.max(np.abs(x))

    # mel-spectrogram
    X = librosa.feature.melspectrogram(y=x,
                                       sr=fs,
                                       n_fft=n_fft,
                                       hop_length=hop_length,
                                       n_mels=n_mels,
                                       fmin=0.0,
                                       fmax=fs / 2,
                                       power=1.0,
                                       htk=True,
                                       norm=None)

    # apply dB normalization
    X = librosa.amplitude_to_db(X)

    return X

Let's compute all mel spectrograms (*this takes a couple of seconds to compute*)

In [ ]:
all_mel_specs = []
for i, fn_wav in enumerate(fn_wav_list):
    all_mel_specs.append(compute_mel_spec_for_audio_file(fn_wav_list[i]))
    
print("We have {} spectrograms of shape {}".format(len(all_mel_specs), all_mel_specs[0].shape))

The final shape of each spectrogram is ```(64, 251)``` and covers 64 mel frequency bands and 251 time frames.

### Convolutional Neural Network (CNN)

In contrast to the previous notebook, where we used a DNN for the animal classification task, we now want to set up a convolutional neural network (CNN) and use the Mel spectrograms as "image-like" input data. The intuition is that the CNN should be able to learn to recognize specific temporal-spectral patterns of each species.

In [ ]:
all_mel_specs = np.array(all_mel_specs)
print(f"Shape of our data tensor : {all_mel_specs.shape}")

The tensor dimensions are:
- batch dimension (25) - we have 25 spectrogram examples
- frequency dimension (64) - our Mel spectrogram has 64 Mel bands
- time dimension (251) - the spectrograms have 251 time frames

**Next Step**: For training the CNN, we would like to have **more examples**. We will randomly cut out 10 segments from each spectrogram (we assume that the typical animal sounds are audible not just once but rather throughout the recording) -> every segment will "inherit" the class label from its original spectrogram

In [ ]:
segment_list = []
segment_file_id = []
segment_class_id = []
segment_spec_id = []

n_spectrograms = all_mel_specs.shape[0]

n_segments_per_spectrogram = 10
segment_length_frames = 100

spec_length_frames = all_mel_specs.shape[2]
max_segment_start_offset = spec_length_frames - segment_length_frames

# iterate over all spectrograms
for i in range(n_spectrograms):
    
    # create [n_segments_per_spectrogram] segments
    for s in range(n_segments_per_spectrogram):
        
        # random segment start frame
        segment_start_frames = int(np.random.rand(1)*max_segment_start_offset)
    
        segment_list.append(all_mel_specs[i, :, segment_start_frames:segment_start_frames+segment_length_frames])
        
        segment_file_id.append(i)
        segment_class_id.append(class_id[i])
        segment_spec_id.append(s)
        
# finally, let's convert our list of spectrogram segments again into a 3D tensor
segment_list = np.array(segment_list)

segment_file_id = np.array(segment_file_id)
segment_file_mod_id = np.mod(segment_file_id, 5)

segment_class_id = np.array(segment_class_id)
segment_spec_id = np.array(segment_spec_id)


print(f"New data tensor shape {segment_list.shape}")   
    

In [ ]:
pl.figure(figsize=(12, 4))
pl.plot(segment_file_id, 'b-', label='segment file ID')
pl.plot(segment_file_mod_id, 'b--', label='segment file ID (per spectrogram)')
pl.plot(segment_class_id, label='segment class ID')
pl.plot(segment_spec_id, label='segment ID')
pl.legend()
pl.xlabel('Segment')
pl.show()

**Obersevation**
- The **segment_file_id** tells us, which of the 25 original spectrograms a segment comes from.
- The **segment_file_mod_id** tells us, which of the 5 example spectrograms (from each animal class), the segment comes from.
- The **class_id** tells us, which of the 5 animal classes a segment relates to.
- The **segment_id** is an index from 0 to 9 (we have 10 segments per spectrogram).

Let's have a closer look into our first dataset file (top) and the segments (below) that we extract.

In [ ]:
pl.figure(figsize=(2.5, 2))
pl.imshow(all_mel_specs[0, :, :], origin="lower", aspect="auto", interpolation="None")
pl.xticks([], [])
pl.yticks([], [])
pl.title('Original spectrogram')
pl.tight_layout()
pl.show()

pl.figure(figsize=(15,5))
ny = 2
nx = int(n_segments_per_spectrogram // ny)
for s in range(n_segments_per_spectrogram):
    pl.subplot(ny, nx, s+1)
    pl.imshow(segment_list[s, :, :], origin="lower", aspect="auto", interpolation="None")
    if s == 0:
        pl.title('Extracted segments')
    pl.xticks([], [])
    pl.yticks([], [])
pl.tight_layout()
pl.show()


**Obersevation**: Our code works. We extracted 10 segments at random time positions. All of these segments show characteristic properties:
  - fundamental & harmonic frequencies
  - time-varying frequency contours
  - background noise

### Dataset split into training and test set

We'll start with the code from the previous notebook. Again, we take the first three files per class as training data and the final two files as test data.

In [ ]:
is_train = np.where(segment_file_mod_id <= 2)[0]
is_test = np.where(segment_file_mod_id >= 3)[0]

print("Our feature matrix is split into {} training examples and {} test examples".format(len(is_train), len(is_test)))

Now that we have splitted our dataset, we can generate the feature matrix and target vectors for the training and test set.

In [ ]:
X_train = segment_list[is_train, :, :]
y_train = segment_class_id[is_train]
X_test = segment_list[is_test, :, :]
y_test = segment_class_id[is_test]

print("Let's look at the dimensions")
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

Let's normalize our feature matrix first, this makes sure that all feature dimensions (columns in the feature matrix) have a mean of 0 and a standard deviation of 1. This makes it easier for the classifier.

In [ ]:
X_train_norm = np.zeros_like(X_train)
X_test_norm = np.zeros_like(X_test)

for i in range(X_train.shape[0]):
    X_train_norm[i, :, :] = StandardScaler().fit_transform(X_train[i, :, :])
    
for i in range(X_test.shape[0]):
    X_test_norm[i, :, :] = StandardScaler().fit_transform(X_test[i, :, :])

### One-hot encoding

Similar to the previous notebook, we have to one-hot-encode our class labels for all segments.



In [ ]:
y_train_transformed = OneHotEncoder(categories='auto', sparse=False).fit_transform(y_train.reshape(-1, 1))
y_test_transformed = OneHotEncoder(categories='auto', sparse=False).fit_transform(y_test.reshape(-1, 1))

### Channel dimension

As a final step, we'll add a fourth ("singleton") dimension to our training and test feature tensors, which mean that we only use one channel.

**Remember**: For image classification tasks, we would use three channels to separately encode the red, green, and blue color channels (RGB channels).

In [ ]:
if len(X_train_norm.shape) == 3:
    X_train_norm = np.expand_dims(X_train_norm, -1)
    X_test_norm = np.expand_dims(X_test_norm, -1)

else:
    print("We already have four dimensions")
    
print(f"Let's check if we have four dimensions. New shapes: {X_train_norm.shape} & {X_test_norm.shape}")


### Convolutional Neural Network

#### Create Model

Let's build our neural network. We will continue to use the [Sequential API](https://keras.io/guides/sequential_model/) for doing this easily. Later, when we use more complex models, we use the [Functional API](https://keras.io/guides/functional_api/) to do this more flexibly.

We will need the following ingredients:
  - the ```Input``` layer defines the input shape that the CNN can expect. This is the shape of the segments in our feature matrix
  - the convolutional layers (```Conv2D```) are the core part of the CNN. They learn specific filters to recognitize the animals based on characteristic patterns in the spectrogram
  - each convolutional layer will be followed by a pooling layer (```MaxPooling2D```) to reduce the spatial dimensions of the output. 
  - finally, we will add a fully connected layer (```Dense```) for the classification output.

We furthermore use the ```Sequential``` model, which allows to easily add more layers to a network.
 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# The input shape is the "time-frequency shape" of our segments + the number of channels
# Make sure to NOT include the first (batch) dimension!
input_shape = X_train_norm.shape[1:]

# Get the number of classes:
n_classes = y_train_transformed.shape[1]

# Define the model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(3,3)))

# 2nd Convolutional Layer
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3,3)))

# 3rd Convolutional Layer
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3,3)))

# Flatten the output to feed into the Dense layer
model.add(Flatten())

# Output layers for classification
model.add(Dense(units=64, activation='relu'))
# Dropout for regularization
model.add(Dropout(0.5))
model.add(Dense(units=n_classes, activation='softmax')) 

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Let's have a look into the architecture of our model.

In [ ]:
model.summary()

Finally, let's compile the model. Then we're ready for model training :)

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

#### Train model

Now we train the model over $50$ iterations using our training data.

In [ ]:
history = model.fit(X_train_norm, y_train_transformed, epochs=50, batch_size=32, verbose=2)

**Observations**:

- We see that the **loss** value is decreasing and at the same time the **accuracy** is increasing, which show us that the model learns to classify the **training data** better and better over the first 50 epochs.

In [ ]:
pl.figure(figsize=(8, 4))
pl.subplot(2,1,1)
pl.plot(history.history['loss'])
pl.ylabel('Loss')
pl.subplot(2,1,2)
pl.plot(history.history['accuracy'])
pl.ylabel('Accuracy (Training Set)')
pl.xlabel('Epoch')
pl.show()


#### Evaluate model

Now we evaluate the model using the **test data** and compute the **accuracy**.

In [ ]:
y_test_pred = model.predict(X_test_norm)

print("Shape of the predictions: {}".format(y_test_pred.shape))

# The model outputs in each row 5 probability values (they always add to 1!) for each class. 
# We want take the class with the highest probability as prediction!

y_test_pred = np.argmax(y_test_pred, axis=1)
print("Shape of the predictions now: {}".format(y_test_pred.shape))

# Accuracy
accuracy = accuracy_score(y_test, y_test_pred)
print("Accuracy score = ", accuracy)

For the **evaluation**, we will compute the **accuracy** and the **confusion matrix**

In [ ]:
pl.figure(figsize=(3,3))
cm = confusion_matrix(y_test, y_test_pred).astype(np.float32)
# normalize to probabilities
for i in range(cm.shape[0]):
    if np.sum(cm[i, :]) > 0:
        cm[i, :] /= np.sum(cm[i, :])  # by dividing through the sum, we convert counts to probabilities
pl.imshow(cm)
ticks = np.arange(5)
pl.xticks(ticks, unique_classes)
pl.yticks(ticks, unique_classes)
pl.show()

**How to further improve this algorithm?**
  - for now, we computed the predictions **per segment**, we can easily aggregate them **to file-level classifications** by **averaging over the frame-level probabilies**
  - change the number and kernel size for the Conv2D layers
  - try to integrate Dropout after the MaxPooling layers as well
  - try to train the model for a longer time (more epochs)
  - try another optimizer (see https://analyticsindiamag.com/guide-to-tensorflow-keras-optimizers/)

In [ ]:
print("Done :)")